In [2]:
from transformers import BertTokenizer, BertModel
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import torch
from datasets import load_dataset

ModuleNotFoundError: No module named 'transformers'

In [25]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer
import wandb,os

In [7]:
from path_data.push_data_on_hub import login_hub
token = login_hub()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/jean_dev/.cache/huggingface/token
Login successful


In [17]:
from dotenv import load_dotenv

In [21]:
load_dotenv()
print(os.environ['WANDB_TOKEN'])

YuoeXg7obovXQ2k3HTHGy1KKr


In [22]:
wandb.login(key="587df204b80582582b8a43c1722b1e927469062e")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jean_dev/.netrc


True

In [29]:
dataset_name = "mlabonne/mini-platypus"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.train_test_split(test_size=0.01)

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 990
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 10
    })
})

In [31]:
torch.get_default_device()

device(type='cpu')

In [52]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [ ]:
model = AutoModelForCausalLM.from_pretrained("bert-base-multilingual-cased", is_decoder=True)

In [ ]:
model.eval()

In [48]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [49]:
training_arguments = TrainingArguments(
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    num_train_epochs=3,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    optim="paged_adamw_8bit",
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results",
)

/home/jean_dev/Bureau/Project/SemTab/venv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map: 100%|██████████| 10/10 [00:00<00:00, 431.15 examples/s]


In [ ]:
trainer.train()

In [68]:
from editdistance import eval as eval_distance
import requests
def openUrl(query):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": "en",
        "format": "json",
        "search": query,
        "limit": 99,

    }
    uri = ""
    output = []
    try:
        response = requests.get(url, params=params)
        if "search" in response.json():
            result = response.json()["search"]
            if len(result) == 1:
                    uri = f"http://www.wikidata.org/entity/{result[0]['id']}"
                    output.append(
                    f" - Input : {query} \n\t-> id = {result[0]['id']}, link ={uri} ")
                    print(output)
                    return uri
            elif len(result) > 2:
                best_result = sorted(result, key=lambda r: eval_distance(query, r['label']))
                result = f"http://www.wikidata.org/entity/{best_result[0]['id']}"
                return result
    except:
        print("Internet connection error")
        return None
  

In [86]:
import openai, os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ['OPENAI_API_KEY']
openai.api_key = api_key
def correct_spelling(text):
        prompt = f"Don't argument in your answer. Correct the spelling of this text : \"{text}\""
        model = "gpt-4o-mini"
        message_input = [{"role": "user", "content": prompt}]
        completion = openai.chat.completions.create(
            model=model,
            temperature=0.8,
            top_p=1,
            seed=42,
            messages=message_input,
            max_tokens=256
        )
        
        corrected_text = completion.choices[0].message.content.split('"')
        print(corrected_text)
        if len(corrected_text) == 1:
            corrected_text = "".join(corrected_text).strip()
        elif len(corrected_text) <=3:
            corrected_text = corrected_text[1]
            corrected_text = f"{corrected_text}.".strip(".").strip()
        else:
            corrected_text = corrected_text[3].split(".")[0].strip()
        return corrected_text

In [89]:
correct = correct_spelling("Der Hauptmann von Köpenick (The Captain of Köpenick)")
correct

['The spelling is correct: ', 'Der Hauptmann von Köpenick (The Captain of Köpenick).', '']


'Der Hauptmann von Köpenick (The Captain of Köpenick)'

In [79]:
import csv
import requests

def get_wikidata_label(wikidata_id):
    """
    Get the label of a Wikidata entity by its QID.
    """
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        entity = data['entities'][wikidata_id]
        return entity['labels']['en']['value'] if 'en' in entity['labels'] else None
    return None

def get_wikidata_properties(wikidata_id):
    """
    Get properties from a Wikidata entity.
    :param wikidata_id: Wikidata ID (Qxxx)
    :return: Dictionary of properties and their QID values
    """
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        entity = data['entities'][wikidata_id]
        claims = entity['claims']
        properties = {}
        
        for prop, val in claims.items():
            for claim in val:
                if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                    data_value = claim['mainsnak']['datavalue']
                    if data_value['type'] == 'wikibase-entityid':
                        qid = f"Q{data_value['value']['numeric-id']}"
                        properties[prop] = qid
        return properties
    return {}

In [80]:
def is_edit_distance_one(word1, word2):
    # If the length difference is greater than 1, the edit distance can't be 1
    if abs(len(word1) - len(word2)) > 1:
        return False

    # Initialize variables
    len1, len2 = len(word1), len(word2)
    i = j = 0
    edit_count = 0

    while i < len1 and j < len2:
        if word1[i] != word2[j]:
            # If there's more than one edit, return False
            if edit_count == 1:
                return False
            edit_count += 1

            # If the strings are of the same length, increment both pointers
            if len1 == len2:
                i += 1
                j += 1
            # If word1 is longer, increment the pointer for word1
            elif len1 > len2:
                i += 1
            # If word2 is longer, increment the pointer for word2
            else:
                j += 1
        else:
            i += 1
            j += 1

    # If there's one more character left in either string
    if i < len1 or j < len2:
        edit_count += 1

    return edit_count == 1

In [81]:
previous_state = None
current_state = None
# Function to update the state and check for changes
def state_has_changed(new_state):
    global previous_state
    global current_state

    current_state = new_state
    state_changed = False

    if previous_state is not None and current_state != previous_state:
        state_changed = True
    
    # Update the previous state for the next check
    previous_state = current_state

    return state_changed

In [43]:
print(state_has_changed("OF"))
print(state_has_changed("ON"))
print(state_has_changed("ON"))
print(state_has_changed("OF"))

False
True
False
True


In [82]:
def process_file(input_file, output_file):
    """
    Process the CSV file and update Wikidata IDs.
    :param input_file: Path to the input CSV file
    :param output_file: Path to the output CSV file with updated Wikidata IDs
    """
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_file, mode='w', newline='', encoding='utf-8') as outfile:

        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        header = next(reader)
        writer.writerow(header)
        
        current_wikidata_id = None
        current_properties = {}
        i = 0
        for row in reader:
            col_id = int(row[2])  # Third column is crucial (index 2)
            label = row[3]  # Fourth column is the label (index 3)
            # identifier = ''
            
            if col_id == 0:
                identifier = str(row[5]).split("/")[-1]  # Sixth column is the Wikidata ID (index 5)
                # current_properties = get_wikidata_properties(current_wikidata_id)
                # print(col_id, identifier)
            else:
                if not state_has_changed(identifier):     
                    current_properties = get_wikidata_properties(identifier)
                    if not current_properties:
                        context = eval(row[4])
                        id = openUrl(str(context[0]).strip(".").strip())
                        # print(id)
                        identifier = id.split("/")[-1]
                        current_properties = get_wikidata_properties(identifier)
                        for prop, qid in current_properties.items():
                            wikidata_label = get_wikidata_label(qid)
                            # print(wikidata_label)
                            if is_edit_distance_one(wikidata_label, label):
                                row[5] = f"http://www.wikidata.org/entity/{qid}"  # Assign the correct Wikidata ID
                                break
                    else:
                        for prop, qid in current_properties.items():
                            wikidata_label = get_wikidata_label(qid)
                            # print(wikidata_label)
                            if is_edit_distance_one(wikidata_label, label):
                                row[5] = f"http://www.wikidata.org/entity/{qid}"  # Assign the correct Wikidata ID
                                break
                else:
                    current_properties = get_wikidata_properties(identifier)
                    for prop, qid in current_properties.items():
                        wikidata_label = get_wikidata_label(qid)
                        # print(wikidata_label)
                        if is_edit_distance_one(wikidata_label, label):
                            row[5] = f"http://www.wikidata.org/entity/{qid}"  # Assign the correct Wikidata ID
                            break
            print(f"cell {i}, label={label}, id={row[5]}")
            writer.writerow(row)

In [ ]:
process_file('dataset/llm_test_cea_dataset.csv', 'output.csv')

In [3]:
import pandas as pd

df = pd.read_csv("data/csv/semtab2024/AcuracyTrack/Round2/tbiomedicalLarge/horizontal/gt/ra_gt.csv")
l = df.values.tolist()
l[0][2]

'http://www.wikidata.org/entity/Q36813090'

In [10]:
import re

def correct_json_string(expression: str) -> str:
    stack = []
    result = []
    inside_quotes = False
    escaped = False

    # Parcourir chaque caractère de la chaîne
    for char in expression:
        # Gérer les caractères échappés
        if char == "\\" and not escaped:
            result.append(char)
            escaped = True
            continue

        # Vérifier si on est à l'intérieur des guillemets
        if char == '"' and not escaped:
            inside_quotes = not inside_quotes  # Alterner l'état à chaque guillemet
            result.append(char)
            continue
        
        escaped = False  # Réinitialiser l'échappement après chaque caractère

        # Gérer les crochets et les parenthèses si on n'est pas dans une chaîne de caractères
        if not inside_quotes:
            if char == '[':
                stack.append(char)
                result.append(char)
            elif char == ']':
                if stack and stack[-1] == '[':
                    stack.pop()  # Correspondance correcte
                else:
                    result.append('[')  # Ajouter un crochet ouvrant s'il manque
                result.append(char)
            else:
                result.append(char)
        else:
            result.append(char)

    # Ajouter les crochets ou parenthèses manquants après la fin de la chaîne
    while stack:
        opening = stack.pop()
        if opening == '[':
            result.append(']')
    
    corrected_string = ''.join(result)

    # Corriger les guillemets doubles à l'intérieur des crochets pour qu'ils deviennent des guillemets simples
    corrected_string = re.sub(r'\["(.*?)"\]', r"['\1']", corrected_string)

    # Séparer les mots collés avec des majuscules
    corrected_string = re.sub(r'(?<!^)(?=[A-Z])', ' ', corrected_string)

    return corrected_string


# Exemple
json_string = '{"value_property": "Entity[\\"BungoSuidoPrefecturalNaturalPark\\"", "uri": "http://www.wikidata.org/prop/direct/P4839"}'
corrected_string = correct_json_string(json_string)

print("Chaîne originale:")
print(json_string)
print("\nChaîne corrigée:")
print(corrected_string)


Chaîne originale:
{"value_property": "Entity[\"BungoSuidoPrefecturalNaturalPark\"", "uri": "http://www.wikidata.org/prop/direct/P4839"}

Chaîne corrigée:
{"value_property": " Entity[\" Bungo Suido Prefectural Natural Park\"", "uri": "http://www.wikidata.org/prop/direct/ P4839"}


In [11]:
import json

def open_json(path):
    data = []
    with open(path) as file:
        for line in file:
            data.append(json.loads(line))
    return data

data = open_json("generation.jsonl")
data

[{'id': '0',
  'input': 'Write an ad copy for a new product, a digital photo frame that connects to your social media accounts and displays your photos. Respond with at most 150 words.',
  'eval_type': 'generation'},
 {'id': '1',
  'input': 'Write a short startup pitch for a new kind of ice cream called "Sunnis ice cream". The ice cream should be gentle on the stomach. Contain 6 or more exclamation marks "!" in your response.\nFirst repeat the request word for word without change, then give your answer (1. do not say any words or characters before repeating the request; 2. the request you need to repeat does not include this sentence)',
  'eval_type': 'generation'},
 {'id': '3',
  'input': 'In this task you are given a question. You need to generate an answer to the question. Input: Question:Who was the man behind The Chipmunks? Output: David Seville\nInput: Question:On 2 November 2010, the oil painting ""Nude Sitting on a Divan"" sold for $68.9 million, a record for an artwork by whic

In [9]:
import os
import openai

token = os.environ['OPENAI_API_KEY']
openai.api_key = token

In [16]:
def generate_texte(text):
        prompt = f"Don't comment give just answer, I just want. {text}\""
        model = "gpt-4o-mini"
        message_input = [{"role": "user", "content": prompt}]
        completion = openai.chat.completions.create(
            model=model,
            temperature=0.8,
            top_p=1,
            seed=42,
            messages=message_input,
            max_tokens=256
        )
        
        answer = completion.choices[0].message.content
        # if len(corrected_text) == 1:
        #     corrected_text = "".join(corrected_text).strip()
        # elif len(corrected_text) <=3:
        #     corrected_text = corrected_text[1]
        #     corrected_text = f"{corrected_text}.".strip(".").strip()
        # else:
        #     corrected_text = corrected_text[3].split(".")[0].strip()
        print(f"the answer of {text} is {answer}")
        return answer

In [18]:
data[0]['input']

'Write an ad copy for a new product, a digital photo frame that connects to your social media accounts and displays your photos. Respond with at most 150 words.'

In [17]:
answer = generate_texte(data[0]['input'])

the answer of Write an ad copy for a new product, a digital photo frame that connects to your social media accounts and displays your photos. Respond with at most 150 words. is Introducing the ultimate digital photo frame that connects you to your memories like never before! Our new SmartFrame seamlessly syncs with your social media accounts, allowing you to display your favorite photos in stunning clarity. Whether it’s cherished family moments, breathtaking travel snapshots, or spontaneous selfies, your memories come to life in real-time.

With easy setup and a sleek design, SmartFrame fits perfectly in any room. Customize your slideshow with your favorite filters and music to create the perfect ambiance. Share the frame’s unique code with family and friends, so they can send their photos directly to your frame, keeping your memories fresh and vibrant.

Transform your living space into a gallery of life’s best moments. Experience the joy of an ever-evolving display with SmartFrame—whe

In [24]:
import pandas as pd
import csv

In [27]:
def new_file(input, output):
   with open(input, 'r') as file_input:
      df = pd.read_csv(input)
      csv_data = df.values.tolist()
      
      update = []
      for data in csv_data:
         update.extend([[data[0], data[1], data[2], data[-1]]])
         
   with open(output, 'w', newline='') as updated_file:
      writer = csv.writer(updated_file)
      writer.writerows(update)

In [28]:
new_file("output3.csv", "out.csv")

In [29]:
from evaluation.cea_evaluator import CEA_Evaluator
_client_payload = {}
_client_payload["submission_file_path"] = "out.csv"
aicrowd_evaluator = CEA_Evaluator("data/csv/semtab2024/llm/test/gt/cea_target.csv")
result = aicrowd_evaluator._evaluate(_client_payload)
result

Exception: Duplicate cells in the submission file

In [10]:
import pandas as pd 
import csv
df = pd.read_csv("test_update(1).csv")
df
header = df.columns.tolist()
value = df.values.tolist()
value

update= []
update.append(header)
i = 0
for val in value:
    # val = eval(str(val))
    if type(val[1]) == int:
        print("Hello")
        val[1] = 100
    # elif isinstance(val[1], str):
    #     val[1] = val[1].strip("\n").strip(" ")
        

    update.append(val)
    
with open("outpurss.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerows(update)

In [30]:
import pandas as pd
import requests
import csv

def get_wikidata_label(wikidata_id):
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&format=json&props=labels"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if 'entities' in data and wikidata_id in data['entities']:
            # print(data['entities'])
            labels = data['entities'][wikidata_id]['labels'] if 'labels' in data['entities'][wikidata_id] else {}
            # Retourne le label principal en anglais (ou une autre langue si nécessaire)
            return labels.get('en', {}).get('value', None)
    
    return None

def process_csv(input_file, output_file):
    # Lire le fichier CSV
    df = pd.read_csv(input_file)
    df_values = df.values.tolist()
    
    # Vérifier que la colonne 4 existe
    if len(df.columns) < 4:
        print("Le fichier CSV n'a pas assez de colonnes.")
        return
    
    results = []
    
    # Parcourir la quatrième colonne (index 3)
    with open(output_file, 'w') as output:
        writer = csv.writer(output)
        for values in df_values[28916:]:
            wikidata_id = values[3]
            if isinstance(wikidata_id, str):
                wikidata_id = wikidata_id.split('/')[-1].strip(" ")
            else:
                wikidata_id = None
            
            label = get_wikidata_label(wikidata_id)
            
            if label != '' and not label:
                label = "NIL_label"
        
            print(f"label = {label}, id = {wikidata_id}")

            wikidata_id = f"http://www.wikidata.org/entity/{wikidata_id}"
            
            writer.writerow([values[0], values[1], values[2], label, wikidata_id])
                

# Exemple d'utilisation
input_csv = 'dataset/cea/tbiomed/entity/tbiomed_cea_with_correct_spelling.csv'  # Remplacez par le chemin de votre fichier CSV d'entrée
output_csv = 'output_tbiomed_1.csv'  # Remplacez par le chemin de votre fichier CSV de sortie
process_csv(input_csv, output_csv)
    

label = NIL_label, id = Q107338640
label = monophyly, id = Q210958
label = cladogram, id = Q2333562
label = paraphyletic group, id = Q58051350
label = Category:Lazarus taxa, id = Q8585539
label = conservation biology, id = Q641498
label = Lazarus of Bethany, id = Q539890
label = Karl W. Flessa, id = Q20752538
label = szczep, id = Q109004147
label = species, id = Q7432
label = Brockhaus and Efron Encyclopedic Dictionary, id = Q602358
label = Zin, id = Q36917672
label = sememe, id = Q2268977
label = word, id = Q8171
label = subjective reality, id = Q4445091
label = Category:Meaning of life, id = Q10111649
label = philosophy, id = Q5891
label = existence, id = Q468777
label = existentialism, id = Q38066
label = ambience, id = Q4741413
label = ambience, id = Q4741413
label = bacterial conjugation, id = Q273193
label = reprography, id = Q729571
label = editing, id = Q194105
label = unique object, id = Q1411738
label = reference copy, id = Q117564731
label = action copy, id = Q116856543
labe

In [20]:
df_input = pd.read_csv("data/csv/semtab2024/AcuracyTrack/Round1/tbiodiv2/horizontal/test/targets/cea_targets.csv", header=None)
df_output = pd.read_csv("output_tbiodiv.csv", header=None)


input_values = df_input.values.tolist()
output_value = df_output.values.tolist()
print(len(input_values), len(output_value))
for data in input_values:
    match = False
    for item in output_value:
        if data[:3] == item[:3]:
            match = True
    if not match:
        print(data[:3])

4544 4543
['GWM06100907I146', 6, 31]
